# Detecting Blindness with Deep Learning
> Using CNNs to detect diabetic retinopathy in retina photos

- toc: true 
- badges: true
- comments: true
- author: Nikita Kozodoi
- categories: [python,deep learning,computer vision]
- image: images/posts/retina.png

# 1. Overview

Can deep learning help to detect blindness?

This blogpost describes the pipeline of the project that develops a convolutional neural network (CNN) for predicting the severity of the diabetic retinopathy based on the patient's retina photograph. The project was completed within the scope of the [Udacity Machine Learning Engineer](https://confirm.udacity.com/LMMJDA7C) nano-degree program and the [Kaggle competition](https://www.kaggle.com/c/aptos2019-blindness-detection/data) hosted by the Asia Pacific Tele-Ophthalmology Society (APTOS).

The blogpost provides a project walkthrough covering the most important modeling steps.
- data exploration and image preprocessing to normalize images from different clinics
- using transfer learning to pre-train CNN on a large data set and fine-tune the model on target domain
- implementing techniques such as learnung rate scheduler, test-time augmentation and others to improve the performance

The modeling is done in `PyTorch`. All Jupyter notebooks and a PDF report are [available on Github](https://github.com/kozodoi/Udacity_Blindness_Detection).

# 2. Motivation

Diabetic retinopathy (DR) is one of the leading causes of vision loss. The World Health Organization reports that more than 300 million people worldwide have diabetes (Wong et al 2016). According to a recent study from International Diabetes Federation, the global prevalence of DR among the individuals with diabetes for the period from 2015 to 2019 was at more than 25% (Thomas et al 2019). The disease prevalence has been rising rapidly in developing countries.

Early detection and treatment are crucial steps towards preventing DR. The screening procedure requires a trained clinical expert to examine the fundus photographs of the patient's retina. This creates delays in diagnosis and treatment of the disease. This is especially relevant for developing countries, which often lack qualified medical stuff to perform the diagnosis. Automated detection of DR can speed up the efficiency and coverage of the screening programs.